In [148]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import date, timedelta, datetime
import time
import random

# create app
sc = SparkSession.builder.appName("cleansingcrimes")\
.config ("spark.sql.shuffle.partitions", "50")\
.config("spark.driver.maxResultSize","5g")\
.config ("spark.sql.execution.arrow.enabled", "true")\
.getOrCreate()

# Extract Json file
df = sc.read.csv('dataset/crimes.csv') 
df.show(5, truncate=False)

+----+--------+-------+----------------+----------------+---------+---------+---------------------------+---------+---------------+---------------------+---------------------+-------+
|_c0 |_c1     |_c2    |_c3             |_c4             |_c5      |_c6      |_c7                        |_c8      |_c9            |_c10                 |_c11                 |_c12   |
+----+--------+-------+----------------+----------------+---------+---------+---------------------------+---------+---------------+---------------------+---------------------+-------+
|null|Crime.ID|Month  |Reported.by     |Falls.within    |Longitude|Latitude |Location                   |LSOA.code|LSOA.name      |Crime.type           |Last.outcome.category|Context|
|1   |null    |2016-04|Cleveland Police|Cleveland Police|-1.238154|54.710767|On or near Greyfriers Court|E01011954|Hartlepool 001A|Anti-social behaviour|null                 |NA     |
|2   |null    |2016-04|Cleveland Police|Cleveland Police|-1.239958|54.710607|On 

In [149]:
# Jadikan row pertama sebagai Kolom
df = df.na.fill(value="No",subset=["_c0"])
row_list = df.select('*').first()
list_column = [x for x in row_list]
df = df.toDF(*list_column)
# drop row pertama
df = df.filter(df.No != "No")
df.show(5, truncate=False)

+---+--------+-------+----------------+----------------+---------+---------+---------------------------+---------+---------------+---------------------+---------------------+-------+
|No |Crime.ID|Month  |Reported.by     |Falls.within    |Longitude|Latitude |Location                   |LSOA.code|LSOA.name      |Crime.type           |Last.outcome.category|Context|
+---+--------+-------+----------------+----------------+---------+---------+---------------------------+---------+---------------+---------------------+---------------------+-------+
|1  |null    |2016-04|Cleveland Police|Cleveland Police|-1.238154|54.710767|On or near Greyfriers Court|E01011954|Hartlepool 001A|Anti-social behaviour|null                 |NA     |
|2  |null    |2016-04|Cleveland Police|Cleveland Police|-1.239958|54.710607|On or near Marshall Close  |E01011954|Hartlepool 001A|Anti-social behaviour|null                 |NA     |
|3  |null    |2016-04|Cleveland Police|Cleveland Police|-1.239958|54.710607|On or nea

In [150]:
# drop Missing Value Crime.ID
df = df.na.drop(subset=["`Crime.ID`"])
df.show(3,truncate=False)
df.filter(col("`Crime.ID`").isNull()).count()

+---+----------------------------------------------------------------+-------+----------------+----------------+---------+---------+-----------------------+---------+---------------+-------------------------+---------------------------------------------+-------+
|No |Crime.ID                                                        |Month  |Reported.by     |Falls.within    |Longitude|Latitude |Location               |LSOA.code|LSOA.name      |Crime.type               |Last.outcome.category                        |Context|
+---+----------------------------------------------------------------+-------+----------------+----------------+---------+---------+-----------------------+---------+---------------+-------------------------+---------------------------------------------+-------+
|12 |cc5ee4533c7729c9248930bbc76a0ad9900deb26594b249c931ecd6b090ed4df|2016-04|Cleveland Police|Cleveland Police|-1.235724|54.71041 |On or near Dobson Place|E01011954|Hartlepool 001A|Criminal damage and arson|Inv

0

In [151]:
# Replace value "NA" to 0 from Context column
df = df.withColumn("Context", when(df["Context"] == "NA", 0).otherwise(df["Context"]))
df.show(2, truncate=False)

+---+----------------------------------------------------------------+-------+----------------+----------------+---------+--------+-----------------------+---------+---------------+-------------------------+---------------------------------------------+-------+
|No |Crime.ID                                                        |Month  |Reported.by     |Falls.within    |Longitude|Latitude|Location               |LSOA.code|LSOA.name      |Crime.type               |Last.outcome.category                        |Context|
+---+----------------------------------------------------------------+-------+----------------+----------------+---------+--------+-----------------------+---------+---------------+-------------------------+---------------------------------------------+-------+
|12 |cc5ee4533c7729c9248930bbc76a0ad9900deb26594b249c931ecd6b090ed4df|2016-04|Cleveland Police|Cleveland Police|-1.235724|54.71041|On or near Dobson Place|E01011954|Hartlepool 001A|Criminal damage and arson|Investi

In [152]:
# Ubah tipe data pada kolom No dan Context dari String menjadi Integer
df.printSchema()
for c in df.columns:    
    int_column = ["No", "Context"]
    if c in int_column:
        df = df.withColumn(c, df[c].cast("integer"))
df.printSchema()

root
 |-- No: string (nullable = false)
 |-- Crime.ID: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- Reported.by: string (nullable = true)
 |-- Falls.within: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- LSOA.code: string (nullable = true)
 |-- LSOA.name: string (nullable = true)
 |-- Crime.type: string (nullable = true)
 |-- Last.outcome.category: string (nullable = true)
 |-- Context: string (nullable = true)

root
 |-- No: integer (nullable = true)
 |-- Crime.ID: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- Reported.by: string (nullable = true)
 |-- Falls.within: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- LSOA.code: string (nullable = true)
 |-- LSOA.name: string (nullable = true)
 |-- Crime.type: string (nullable = true)
 |-- Last